In [1]:
#check the direction

In [2]:
import os
print(os.getcwd())

C:\Program Files\Microsoft VS Code


In [3]:
import os

# Change to your project directory (update the path as needed)
os.chdir(r"H:\Work_With_MZ\LC_Classification_Code")

# Verify the change
print(os.getcwd())

H:\Work_With_MZ\LC_Classification_Code


In [4]:
import xarray as xr
import pandas as pd
import pandas as gpd
import regionmask
import numpy as np
import geopandas
import xarray as xr
import xarray as xr
import seaborn as sns
import rioxarray
from shapely.geometry import mapping
import netCDF4
import netCDF4 as nc
from PIL import Image
import os
from netCDF4 import Dataset
import geopandas as gpd
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from scipy.stats import linregress
#from xarrayMannKendall import MannKendall
import pymannkendall as mk
import xarray as xr
import numpy as np
from scipy.stats import kendalltau

In [9]:
import os
import rasterio
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
import geopandas as gpd
import numpy as np
import xarray as xr

# Paths
tif_folder = "tif_files"
bands = ["B2.tif", "B3.tif", "B4.tif", "B8.tif", "B11.tif", "B12.tif"]
shapefile = "Province_Sofian/Export_Output.shp"

# Reference band (for resolution, transform, crs)
ref_path = os.path.join(tif_folder, "B2.tif")
with rasterio.open(ref_path) as ref:
    ref_crs = ref.crs
    ref_transform = ref.transform
    ref_shape = (ref.height, ref.width)
    ref_profile = ref.profile

# Read and reproject shapefile to raster CRS
gdf = gpd.read_file(shapefile)
if gdf.crs != ref_crs:
    gdf = gdf.to_crs(ref_crs)
roi_geom = [gdf.geometry.unary_union]

# Prepare output arrays
band_arrays = []
band_names = []

for band in bands:
    band_path = os.path.join(tif_folder, band)
    with rasterio.open(band_path) as src:
        # Read and resample to reference grid if needed
        data = src.read(1)
        if (src.transform != ref_transform) or (src.shape != ref_shape) or (src.crs != ref_crs):
            data_resampled = np.empty(ref_shape, dtype=data.dtype)
            reproject(
                source=data,
                destination=data_resampled,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref_transform,
                dst_crs=ref_crs,
                resampling=Resampling.bilinear
            )
        else:
            data_resampled = data

        # Mask (clip) with ROI
        # To use rasterio.mask, we need a dataset object, so use MemoryFile
        with rasterio.io.MemoryFile() as memfile:
            with memfile.open(
                driver='GTiff',
                height=ref_shape[0],
                width=ref_shape[1],
                count=1,
                dtype=data_resampled.dtype,
                crs=ref_crs,
                transform=ref_transform,
            ) as dataset:
                dataset.write(data_resampled, 1)
                out_image, out_transform = mask(
                    dataset,
                    roi_geom,
                    crop=True,
                    filled=True,
                    nodata=0,
                    all_touched=True
                )
        band_arrays.append(out_image[0])
        band_names.append(band.split('.')[0])

# Stack and save as NetCDF
stacked = np.stack(band_arrays, axis=0)  # shape: (bands, y, x)
da = xr.DataArray(
    stacked,
    dims=("band", "y", "x"),
    coords={"band": band_names}
)
ds = xr.Dataset({"sentinel2": da})
ds.to_netcdf("sentinel2_clipped.nc")
print("Saved as sentinel2_clipped.nc")

Saved as sentinel2_clipped.nc
